# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097012


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:15,  2.71s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:03,  2.35s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:41,  1.66s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:27,  1.18s/it]

...

Rendering models:  63%|██████▎   | 19/30 [00:07<00:02,  4.04it/s]

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  5.45it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.99it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00,  8.33it/s]

not-logged-in-3b0b929f4475fd276561    0.004006
not-logged-in-3b0b929f4475fd276561    0.000318
Jnursssmith5263                       0.001918
not-logged-in-673e227f920b24558cc1    0.000552
rupertcalvert                         0.008242
ElisabethB                            0.000325
PRivate_pAuLa                         0.000581
ellivc                                0.000321
not-logged-in-906d0937c18c013d2c32    0.002880
JoshMcflurry                          0.182541
Jessica_Korkmaz                       0.003318
Heckrex                               0.004125
not-logged-in-5f30b2bb9b7f9d49da41    0.388384
DarioB                                0.000538
pangeli5                              0.005987
peakscience7                          0.001608
not-logged-in-3ae935e3afaee843f530    0.000682
not-logged-in-db4d0b52f3105920643a    0.000232
Lavadude                              0.001690
ts135992                              0.000184
not-logged-in-3591469ee2ac3f75cbf7    0.002993
ZoshiePoshie 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())